In [ ]:
import base64
import pickle
import os
import pandas as pd
from io import BytesIO
import requests
from ipywidgets import widgets
from IPython.display import HTML, SVG, Image, Javascript, display
from threading import Thread
from datetime import datetime
import time

In [ ]:
pd.__version__

In [ ]:
from rdkit import Chem, RDConfig
from rdkit.Chem.Draw import rdMolDraw2D, IPythonConsole, MolsToGridImage

In [ ]:
os.getcwd()

In [ ]:
with open("JupyterDrawOptions.json") as hnd:
    rdMolDraw2D.UpdateMolDrawOptionsFromJSON(IPythonConsole.drawOptions, hnd.read())

In [ ]:
IPythonConsole.ipython_showProperties = True
IPythonConsole.ipython_useSVG = False

In [ ]:
from rdkit.Chem import PandasTools

In [ ]:
bilastine_pubchem_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/bilastine/SDF"

In [ ]:
response = requests.get(bilastine_pubchem_url, allow_redirects=True)

In [ ]:
assert response.status_code == 200

In [ ]:
bilastine_sdf = response.content

In [ ]:
with BytesIO(bilastine_sdf) as buf:
    with Chem.ForwardSDMolSupplier(buf) as suppl:
        bilastine = next(suppl)

In [ ]:
bilastine

In [ ]:
with BytesIO(bilastine_sdf) as buf:
    df = PandasTools.LoadSDF(buf)

In [ ]:
romol = df.ROMol
df.drop(columns=["ROMol"], inplace=True)
df.insert(0, "ROMol", romol)

The size of the PNG molecule depends on the size of the column label rather than on `PandasTools.molSize`:

In [ ]:
df

In [ ]:
PandasTools.molRepresentation = "svg"

This is not the case for SVG, but you may notice that the molecule in the `pandas` DataFrame does not honor the custom `drawOptions` that were set in `IPythonConsole`:

In [ ]:
df

`PandasTools.LoadSDF()` automatically calls `RenderImagesInAllDataFrames`, so let's revert that to see what happens when the global flag is not set.

As expected, now the molecule is not shown.

In [ ]:
PandasTools.RenderImagesInAllDataFrames(False)

In [ ]:
df2 = pd.DataFrame({"mol": [bilastine]})

In [ ]:
df2

The DataFrame should be displayed with molecules after calling `ChangeMoleculeRendering` on it, but it is not:

In [ ]:
PandasTools.ChangeMoleculeRendering(df2)

In [ ]:
df2

The notebook can only be displayed by doing this:

In [ ]:
HTML(df2.to_html(notebook=True))

The visualization only works propertly when globally enabling molecule visualization on all DataFrames:

In [ ]:
df3 = pd.DataFrame({"mol": [bilastine]})

In [ ]:
df3

In [ ]:
PandasTools.RenderImagesInAllDataFrames()

In [ ]:
df3

All of these are minor issues. Let's look at a much bigger one.

The 2022.03 release (and many others that preceded it: I think this problem has affected `PandasTools` for a long time) used to patch <a target="_blank" href="https://github.com/rdkit/rdkit/blob/92acaa51908d5f42fb30555b94604a92786c88e9/rdkit/Chem/PandasTools.py#L469">`Chem.Mol.__str__`</a> to visualize molecules as images rather as strings in `pandas` DataFrames:<br>

In turn, `PrintAsBase64PNGString` calls <a target="_blank" href="https://github.com/rdkit/rdkit/blob/92acaa51908d5f42fb30555b94604a92786c88e9/rdkit/Chem/Draw/__init__.py#L447">`Draw._moltoSVG`</a> or <a target="_blank" href="https://github.com/rdkit/rdkit/blob/92acaa51908d5f42fb30555b94604a92786c88e9/rdkit/Chem/Draw/__init__.py#L490">`Draw._moltoimg`</a>, depending on whether an SVG or a PNG image were requested.<br>
Both will involve a 2D molecule layout, which will take a non-trivial amount of time if the molecule do not have coordinates.

I will now analyze the impact that this seemingly harmless patch may have on Jupyter Notebook performance.

In [ ]:
def counter_box():
    """Returns an HTML IPyWidget displaying a counter"""
    return widgets.HTML(value=f"<code>0</code>").add_class("Chem_Mol__str__counter")

In [ ]:
def reposition_counter_box(widget):
    """Some JS hackery to display the counter box on top of the Notebook rather than inlined"""
    display(Javascript("""
const div = document.querySelector('div[class*=Chem_Mol__str__counter]');
const label = div.querySelector('label');
const titleDiv = document.createElement('div');
titleDiv.style = 'margin-bottom: 4px';
titleDiv.innerHTML = '<code>Chem.Mol.__str.__</code> call counter';
const countDiv = div.querySelector('div[class=widget-html-content]');
countDiv.style.textAlign = 'right';
div.insertBefore(titleDiv, label);
const TOP_PADDING = 120;
const RIGHT_PADDING = 100;
const viewPortWidth = window.innerWidth || document.body.clientWidth;
div.style = `display: block; position: absolute; ` +
            `border: 2px solid black; background-color: white; ` +
            `padding: 8px; top: ${TOP_PADDING}px; z-index: 9999;`;
document.body.appendChild(div);
const rect = div.getBoundingClientRect();
const left = Math.round(viewPortWidth - rect.width - RIGHT_PADDING);
div.style.left = `${left}px`;
"""))

In [ ]:
counter_widget = counter_box()

In [ ]:
counter_widget

In [ ]:
reposition_counter_box(counter_widget)

We patch `Chem.Mol.__str__` to show that it gets called for all variables defined in the Jupyter Notebook after each cell is executed:

In [ ]:
def new_str_method(x):
    """Increment call count, then call the original method"""
    new_str_method.__call_count += 1
    return orig_str_method(x)

In [ ]:
# patch Chem.Mol.__str__
# (which had been previously patched by PandasTools)
if not hasattr(Chem.Mol.__str__, "__call_count"):
    orig_str_method = Chem.Mol.__str__
    Chem.Mol.__str__ = new_str_method
    setattr(new_str_method, "__call_count", 0)

In [ ]:
def update_counter_box(counter_widget, new_str_method):
    """Update the counter box every UPDATE_INTERVAL_SECONDS"""
    UPDATE_INTERVAL_SECONDS = 1
    while not hasattr(update_counter_box, "stop"):
        counter_widget.value = f"<code>{new_str_method.__call_count}</code>"
        time.sleep(UPDATE_INTERVAL_SECONDS)

We start a background thread that updates the counter box every second:

In [ ]:
t = Thread(target=update_counter_box, args=(counter_widget, new_str_method), daemon=True)

In [ ]:
t.start()

We load 5000 molecules:

In [ ]:
smi5000 = os.path.join(RDConfig.RDDataDir, "NCI", "first_5K.smi")

In [ ]:
assert os.path.exists(smi5000)

In [ ]:
with Chem.SmilesMolSupplier(smi5000, delimiter="\t") as suppl:
    mols = [mol for mol in suppl if mol]

In [ ]:
len(mols)

Now for each of the 4991 molecules we define a variable in the Notebook's namespace:

In [ ]:
for i in range(len(mols)):
    exec(f"mol{i} = mols[{i}]")

Looking at the counter box, you may see that the counter was incremented for each molecule added to the Notebook as a separate variable.

Now executing any cell takes several seconds, because the coordinates of ~5000 molecules need to be rebuilt in the background:

In [ ]:
start = datetime.now()

In [ ]:
(datetime.now() - start).seconds

In [ ]:
df4991 = pd.DataFrame({"mols": mols})

In [ ]:
df4991

Shortly, the notebook becomes so slow that it is barely usable, because `Chem.Mol.__str__` is too computationally intensive to be called so often.

Even switching tabs Jupyter Lab tabs triggers a large number of calls.

In [ ]:
setattr(update_counter_box, "stop", True)
counter_widget.close()